①将同age、gender、education的搜索内容全部合并
②在合并内容中，随机选取一定长度的搜索内容，作为样本的重采样
③最终选取出20000条左右样本时候结果比较理想

In [ ]:
import pandas as pd
import numpy as np
import random
np.random.seed(1)

train = pd.read_table('E:/BaiduYunDownload/sougou2/user_tag_query.10W.TRAIN',encoding = 'gb18030',header = None,sep = '\n',names = ['col'])
train['ID'] = train['col'].apply(lambda x:x.split('\t')[0])
train['age'] = train['col'].apply(lambda x:x.split('\t')[1])
train['Gender'] = train['col'].apply(lambda x:x.split('\t')[2])
train['Education'] =  train['col'].apply(lambda x:x.split('\t')[3])

train['Query'] = train['col'].apply(lambda x:','.join(x.split('\t')[4:-1]))

train = train.drop(list(train[(train['age']=='0')|(train['Gender']=='0')|(train['Education']=='0')].index),axis = 0)

train_new = train.groupby(['age','Gender','Education'],as_index = False).apply(lambda x:x['Query'].str.cat(sep=','))

In [ ]:
df = pd.DataFrame(train_new)
c = train.groupby(['age','Gender','Education'],as_index = False).count()

In [ ]:
def random_query(data,age,gender,edu):
    L = []
    df = pd.DataFrame()
    count1 = int(c[(c['age']==str(age))&(c['Gender']==str(gender))&(c['Education']==str(edu))]['ID'])
    count = count1/50 #if count1>100 else count1
    query = (data.loc[str(age),str(gender),str(edu)].values[0]).split(',')
    for i in range(count):
        L.append(random.sample(query,random.randint(18,len(query))))
    df['Query'] = L
    df['age'] = age
    df['Gender'] = gender
    df['Education'] = edu
    yield df

In [ ]:
T = pd.DataFrame()
for age in range(1,7):
    for gender in range(1,2):
        for edu in range(1,7):
            T = T.append(random_query(df,age,gender,edu).next())
T.to_csv('E:/BaiduYunDownload/sougou2/TT/user_tag_T60.csv',encoding = 'gb18030')